# 100% local Agentic RAG

In [1]:
import warnings
from crewai import Agent, Crew, Task, LLM, Process
from src.agentic_rag.tools.custom_tool import DocumentSearchTool
from src.agentic_rag.tools.custom_tool import FireCrawlWebSearchTool

warnings.filterwarnings("ignore")

/Users/akshaypachaar/miniconda3/envs/env_crewai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup LLM


In [2]:
llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"
)

### Setup Tools

In [3]:
pdf_tool = DocumentSearchTool(file_path='/Users/akshaypachaar/Eigen/ai-engineering/agentic_rag/knowledge/dspy.pdf')

Make sure you have the API key for FireCrawl in your environment variables.

In [4]:
web_search_tool = FireCrawlWebSearchTool()

### Agents

In [5]:
retriever_agent = Agent(
    role="""Retrieve relevant information to answer the user query: {query}""",
    goal="""Retrieve the most relevant information from the available sources 
            for the user query: {query}, always try to use the pdf search tool first. 
            If you are not able to retrieve the information from the pdf search tool 
            then try to use the web search tool.""",
    backstory="""You're a meticulous analyst with a keen eye for detail. 
                You're known for your ability understand the user query: {query} 
                and retrieve knowlege from the most suitable knowledge base.""",
    verbose=True,
    tools=[
        pdf_tool,
        web_search_tool
    ],
    # llm=llm
)

response_synthesizer_agent = Agent(
    role="""Response synthesizer agent for the user query: {query}""",
    goal="""Synthesize the retrieved information into a concise and coherent response 
            based on the user query: {query}. If you are not able to retrieve the 
            information then respond with "I'm sorry, I couldn't find the information 
            you're looking for.""",
    backstory="""You're a skilled communicator with a knack for turning complex 
                information into clear and concise responses.""",
    verbose=True,
    # llm=llm
)


### Tasks

In [6]:
retrieval_task = Task(
    description="""Retrieve the most relevant information from the available 
                   sources for the user query: {query}""",
    expected_output="""The most relevant information in form of text as retrieved
                       from the sources.""",
    agent=retriever_agent
)

response_task = Task(
    description="""Synthesize the final response for the user query: {query}""",
    expected_output="""A concise and coherent response based on the retrieved infromation
                       from the right source for the user query: {query}. If you are not 
                       able to retrieve the information then respond with 
                       I'm sorry, I couldn't find the information you're looking for.""",
    agent=response_synthesizer_agent
)

### Initialize Crew

In [7]:
crew = Crew(
			agents=[retriever_agent, response_synthesizer_agent], 
			tasks=[retrieval_task, response_task],
			process=Process.sequential,
			verbose=True,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)

### Kickoff Crew

In [10]:
result = crew.kickoff(inputs={"query": "When is Australian open 2025 happening?"})

# Agent: Retrieve relevant information to answer the user query: When is Australian open 2025 happening?
## Task: Retrieve the most relevant information from the available 
                   sources for the user query: When is Australian open 2025 happening?


# Agent: Retrieve relevant information to answer the user query: When is Australian open 2025 happening?
## Thought: I will start by searching the document for information regarding the dates of the Australian Open in 2025.
## Using tool: DocumentSearchTool
## Tool Input: 
"{\"query\": \"Australian Open 2025 dates\"}"
## Tool Output: 
02406, 2022.


___
 In International Conference on Machine
Learning, pp.
___
 arXiv preprint arXiv:2305.03495, 2023.

Peng Qi, Xiaowen Lin, Leo Mehr, Zijian Wang, and Christopher D. Manning. Answering complex
In Proceedings of the 2019 Con-

___
Takuya Akiba, Shotaro Sano, Toshihiko Yanase, Takeru Ohta, and Masanori Koyama. Optuna:
In Proceedings of the 25th ACM
A next-generation hyperparameter opt

In [11]:
print(result)

The Australian Open 2025 will take place from January 6 to January 26, 2025, at Melbourne Park in Melbourne, Australia.
